# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Acarau,-2.8856,-40.1200,25.99,80,30,8.68,BR,1690407795
1,1,Margate,51.3813,1.3862,15.17,93,100,4.02,GB,1690407883
2,2,Stanley,54.8680,-1.6985,13.62,89,40,1.03,GB,1690407709
3,3,Grytviken,-54.2811,-36.5092,-0.02,98,100,4.94,GS,1690407883
4,4,Chadron,42.8294,-102.9999,37.82,15,0,4.63,US,1690407884


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
#%%capture --no-display

# Configure the map plot
vacation_map=city_data_df.hvplot.points("Lng","Lat",geo=True,tiles="OSM",frame_width=700,frame_height=500, size="Humidity",color="City")

# Display the map
vacation_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [42]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather=city_data_df.loc[(city_data_df["Max Temp"]<=32)&(city_data_df["Max Temp"]>=13)&(city_data_df["Humidity"]<=40)&(city_data_df["Wind Speed"]<=3),:]


# Drop any rows with null values
ideal_weather_data=ideal_weather.dropna()

# Display sample data
ideal_weather_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,Korla,41.7597,86.1469,27.94,29,0,0.92,CN,1690407896
45,45,Klyuchi,52.2667,79.1667,22.52,36,100,2.94,RU,1690407907
83,83,Altai,46.3722,96.2583,14.67,30,95,0.74,MN,1690407928
89,89,Tympaki,35.0729,24.7685,27.98,36,0,0.88,GR,1690407931
143,143,Howard Springs,-12.4970,131.0470,18.95,35,79,2.54,AU,1690407954
204,204,Vernon,50.2581,-119.2691,27.18,28,50,2.06,CA,1690407981
208,208,Happy Valley-Goose Bay,53.3168,-60.3315,26.17,36,20,2.57,CA,1690407915
215,215,Balsas,-7.5325,-46.0356,28.62,31,8,2.18,BR,1690407783
336,336,Aydun,32.5072,35.8575,25.14,32,2,1.73,JO,1690408050
341,341,Hotan,37.0997,79.9269,26.59,32,3,1.22,CN,1690408052


### Step 3: Create a new DataFrame called `hotel_df`.

In [43]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_weather_data[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
34,Korla,CN,41.7597,86.1469,29,
45,Klyuchi,RU,52.2667,79.1667,36,
83,Altai,MN,46.3722,96.2583,30,
89,Tympaki,GR,35.0729,24.7685,36,
143,Howard Springs,AU,-12.4970,131.0470,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [46]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
          'filter':'',\
          'bias':'',\
          'apiKey':geoapify_key,\
          'limit':1}

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)
    print(name_address) 

    # Convert the API response to JSON format
    name_address_json = name_address.json()
    print(name_address_json)

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        #If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


<Response [200]>
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Silver Star Hotel', 'country': 'China', 'country_code': 'cn', 'region': 'Bayingolin Mongol Autonomous Prefecture', 'state': 'Xinjiang', 'county': 'Korla City', 'city': 'Sayibage Subdistrict', 'postcode': '841000', 'street': '人民东路', 'lon': 86.15030915799298, 'lat': 41.7621635, 'formatted': 'Silver Star Hotel, 841000 China, Xinjiang, Korla City, Sayibage Subdistrict, 人民东路', 'address_line1': 'Silver Star Hotel', 'address_line2': '841000 China, Xinjiang, Korla City, Sayibage Subdistrict, 人民东路', 'categories': ['accommodation', 'accommodation.hotel'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Silver Star Hotel', 'osm_id': 380337334, 'tourism': 'hotel', 'osm_type': 'w'}}, 'distance': 344, 'place_id': '51340bb3639e89554059c29

,City,Country,Lat,Lng,Humidity,Hotel Name
34,Korla,CN,41.7597,86.1469,29,Silver Star Hotel
45,Klyuchi,RU,52.2667,79.1667,36,No hotel found
83,Altai,MN,46.3722,96.2583,30,Altai hotel 40000-100000T
89,Tympaki,GR,35.0729,24.7685,36,Sifogiannis
143,Howard Springs,AU,-12.4970,131.0470,35,No hotel found
204,Vernon,CA,50.2581,-119.2691,28,Okanagan Royal Park Inn
208,Happy Valley-Goose Bay,CA,53.3168,-60.3315,36,Hotel North Two
215,Balsas,BR,-7.5325,-46.0356,31,Imperial Hotel
336,Aydun,JO,32.5072,35.8575,32,ajnadeen
341,Hotan,CN,37.0997,79.9269,32,Hua Yi International Hotel (accepts foreigners)


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [48]:
%%capture --no-display

# Configure the map plot
hotel_map=hotel_df.hvplot.points("Lng","Lat",geo=True,tiles="OSM",frame_width=700,frame_height=500, size="Humidity",color="City",hover_cols=["Hotel Name","Country"])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)